In [1]:
#GPU Reset if needed

from numba import cuda 
device = cuda.get_current_device()
device.reset()

In [2]:
# Parameters

base_directory = "i:/image_classify/" # base directory
image_directory = base_directory + "dataset_cleaned_part7" # image directory
image_size  = (224, 224) # preprocessing image size for model input to match with input_tensor in the model
classify_names = ["type1_male", "type2_male", "type3_male", "type4_male","type1_female", "type2_female", "type3_female", "type4_female"] # class names to be classified into (same as sub folder names with images in different classes)

batch_size = 25 # number of images per batch to be processed

# unused paramaters
#shuffle_value = True
#seed_value = 1337
#epochs = 1

In [3]:
# Change to base directory

import os
os.chdir(base_directory)
print("base directory : " + os.getcwd())

base directory : i:\image_classify


In [4]:
# Import Tensorflow and related libraries

import tensorflow

from tensorflow.keras.models import load_model

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input

print("TensorFlow version : " + tensorflow.__version__)

TensorFlow version : 2.9.0-dev20220102


In [5]:
# Display available GPUs

physical_devices = tensorflow.config.list_physical_devices('GPU')
for gpu in physical_devices:
    print("Name:", gpu.name, "  Type:", gpu.device_type)

Name: /physical_device:GPU:0   Type: GPU


In [6]:
# Enable Memory Growth

if physical_devices != []:
    tensorflow.config.experimental.set_memory_growth(physical_devices[0], True)
    physical_devices[0]

In [7]:
# Create pipeline to load images

val_ds = tensorflow.keras.preprocessing.image_dataset_from_directory(
    image_directory,
    labels='inferred', 
    label_mode='int', 
    class_names=classify_names,    
    #validation_split=0.3,
    #subset="validation",
    #shuffle=shuffle_value,
    #seed=seed_value,
    image_size=image_size,
    batch_size=batch_size,
)

val_ds = val_ds.prefetch(buffer_size=32)

Found 9359 files belonging to 8 classes.


In [8]:
# load the saved model

saved_model = load_model('classify_once.h5')

# create a model with inpots and outputs to evaluate
input_tensor = Input((224, 224, 3))
model = Model(inputs=input_tensor, outputs=saved_model(input_tensor))
model._name = "verification"
model.summary()


Model: "verification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 classify_once (Functional)  (None, 8)                 26743688  
                                                                 
Total params: 26,743,688
Trainable params: 12,087,304
Non-trainable params: 14,656,384
_________________________________________________________________


In [9]:
# compile the model

model.compile(
    optimizer=tensorflow.keras.optimizers.Adam(1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)

In [10]:
# evaluate the model with test images

print("Evaluate")
result = model.evaluate(val_ds)
dict(zip(model.metrics_names, result))


Evaluate
375/375 [==============================] - 89s 215ms/step - loss: 0.9522 - accuracy: 0.7821


{'loss': 0.9521716833114624, 'accuracy': 0.7821348309516907}